# Function Wrapper `utils.wrap`

Suppose we have a predicate `lower_case : str -> str` that returns the upper-case
version of a string.

When we apply `path` to a JSON-like object, we get a list of `PathValue` objects,
each of which contains a path and a value, e.g. when we apply `path` to the
arguments `**.emails.()` and `obj` given by

```python
obj =  {
    'name': 'Alice',
    'roles': ['admin', 'user'],
    'test': {
        'test2': {
        'test3': {
            'emails': ['AliceDeep@example.com', 'JoeDeep@test.com']
        }
        }
    },
    'profile': {
        'age': 30,
        'emails': ['alice@example.com', 'alice.work@example.com']
    }
}
```

we get the result

```python
[PathValue(value=AliceDeep@example.com, path=['$', 'test', 'test2', 'test3', 'emails', '[0]']),
 PathValue(value=JoeDeep@test.com, path=['$', 'test', 'test2', 'test3', 'emails', '[1]']),
 PathValue(value=alice@example.com, path=['$', 'profile', 'emails', '[0]']),
 PathValue(value=alice.work@example.com, path=['$', 'profile', 'emails', '[1]'])]
 ```

 Now, `lower_case` doesn't work on  a list of `PathValue` objects, it works on a single
 `str` argument. The purpose of `utils.wrap` is to create a new function
`utils.wrap(lower_case) : List[PathValue] -> List[OpValue]` that applies `lower_case` to each
`PathValue` object in the list:

```python
[OpValue(value=ali


In [1]:
from typing import Any, List, Union
import logging
import re
from pprint import pprint
from jaf import path, utils
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

In [2]:
data2 = [
   {
      'name': 'Alice',
      'roles': ['admin', 'user'],
      'test': {
         'test2': {
            'test3': {
               'emails': ['alicedeep@example.com', 'joedeep@test.com']
            }
         }
      },
      'profile': {
         'age': 30,
         'emails': ['alice@example.com', 'alice.work@example.com']
      }
   },
   {
      'name': 'Bob',
      'roles': ['user'],
      'test': {
         'test2': {
            'test3': {
               'emails': ['bob@mymail.com']
            }
         }
      }
   }
]

data = data2[0]

In [9]:
vals = path.path_values(path='**.emails.*', obj=data)
pprint(vals)

[PathValue(value=alicedeep@example.com, path=['$', 'test', 'test2', 'test3', 'emails', '[0]']),
 PathValue(value=joedeep@test.com, path=['$', 'test', 'test2', 'test3', 'emails', '[1]']),
 PathValue(value=alice@example.com, path=['$', 'profile', 'emails', '[0]']),
 PathValue(value=alice.work@example.com, path=['$', 'profile', 'emails', '[1]'])]


In [12]:
f = utils.wrap(func=lambda x, obj: x.lower(), func_name="lower-case")
results = f(args=vals, obj=data)
pprint(results)

DEBUG:jaf.utils:Applying lower-case to [PathValue(value=alicedeep@example.com, path=['$', 'test', 'test2', 'test3', 'emails', '[0]']), PathValue(value=joedeep@test.com, path=['$', 'test', 'test2', 'test3', 'emails', '[1]']), PathValue(value=alice@example.com, path=['$', 'profile', 'emails', '[0]']), PathValue(value=alice.work@example.com, path=['$', 'profile', 'emails', '[1]'])]
DEBUG:jaf.utils:Combinations for 'lower-case': [('alicedeep@example.com', 'joedeep@test.com', 'alice@example.com', 'alice.work@example.com')]
ERROR:jaf.utils:Error in 'lower-case': <lambda>() got multiple values for argument 'obj'


[]


In [5]:
def length(x, obj):
   if isinstance(x, (str, list)):
      return len(x)
   return -1


logger.setLevel(logging.DEBUG)

f1 = utils.wrap(func=lambda x, obj: x.lower())
results1 = f1(results, obj=data)
f2 = utils.wrap(func=length)
results2 = f2(results1, obj=data)
pprint(results2)

DEBUG:jaf.utils:Applying <lambda> to []
DEBUG:jaf.utils:Combinations for '<lambda>': [()]
ERROR:jaf.utils:Error in '<lambda>': <lambda>() missing 1 required positional argument: 'x'
DEBUG:jaf.utils:Applying length to []
DEBUG:jaf.utils:Combinations for 'length': [()]
ERROR:jaf.utils:Error in 'length': length() missing 1 required positional argument: 'x'


[]


In [6]:
def two(x, obj):
   return [x * 2, x + 2]

f3 = utils.wrap(func=two)
results3 = f3(results2, obj=data)
pprint(results3)

DEBUG:jaf.utils:Applying two to []
DEBUG:jaf.utils:Combinations for 'two': [()]
ERROR:jaf.utils:Error in 'two': two() missing 1 required positional argument: 'x'


[]


In [7]:
# let's have two arguments

def add(x, y, obj):
   return x + y

f4 = utils.wrap(func=add)
results4 = f4(results3[0][0], results3[0][1], obj=data)

IndexError: list index out of range